In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install pm4py

In [9]:
!ls '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI'


BPIC12.xes     BPIC13_i.xes	       RTFMP.xes
BPIC13_cp.xes  BPI_Challenge_2012.xes  SEPSIS.xes


In [0]:
#Importing library to read log file
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.exporter.csv import factory as csv_exporter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.objects.log.exporter.xes import factory as xes_exporter
from pm4py.algo.filtering.pandas.variants import variants_filter

from datetime import datetime
import pandas as pd
import operator
import re

In [0]:
#Reading XES fileafter 
filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI/SEPSIS.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need10.csv"
csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis = pd.read_csv(fileName)

# Filtering only three columns
df_clean = df_sepsis.loc[:,['case:concept:name','concept:name','time:timestamp']]

In [0]:
df_clean["case:concept:name"].fillna("KMM",inplace=True)
#len(df_clean))

In [53]:
df_clean["case:concept:name"].isnull().sum()

0

In [63]:
df_clean.head(30)

case:concept:name      concept:name             time:timestamp
0                  A   ER Registration  2014-10-22 11:15:41+02:00
1                  A        Leucocytes  2014-10-22 11:27:00+02:00
2                  A               CRP  2014-10-22 11:27:00+02:00
3                  A        LacticAcid  2014-10-22 11:27:00+02:00
4                  A         ER Triage  2014-10-22 11:33:37+02:00
5                  A  ER Sepsis Triage  2014-10-22 11:34:00+02:00
6                  A         IV Liquid  2014-10-22 14:03:47+02:00
7                  A    IV Antibiotics  2014-10-22 14:03:47+02:00
8                  A      Admission NC  2014-10-22 14:13:19+02:00
9                  A               CRP  2014-10-24 09:00:00+02:00
10                 A        Leucocytes  2014-10-24 09:00:00+02:00
11                 A        Leucocytes  2014-10-26 08:00:00+01:00
12                 A               CRP  2014-10-26 08:00:00+01:00
13                 A        Leucocytes  2014-10-28 08:00:00+01:00
14                 A               CRP  2014-10-28 08:00:00+01:00
15                 A               CRP  2014-10-30 08:00:00+01:00
16                 A        Leucocytes  2014-10-30 08:00:00+01:00
17                 A        Leucocytes  2014-10-31 08:00:00+01:00
18                 A               CRP  2014-10-31 08:00:00+01:00
19                 A               CRP  2014-11-02 08:00:00+01:00
20                 A        Leucocytes  2014-11-02 08:00:00+01:00
21                 A         Release A  2014-11-02 15:15:00+01:00
22                 B   ER Registration  2014-12-21 11:04:24+01:00
23                 B         ER Triage  2014-12-21 11:17:19+01:00
24                 B               CRP  2014-12-21 11:36:00+01:00
25                 B        LacticAcid  2014-12-21 11:36:00+01:00
26                 B        Leucocytes  2014-12-21 11:36:00+01:00
27                 B  ER Sepsis Triage  2014-12-21 12:15:45+01:00
28                 B         IV Liquid  2014-12-21 12:33:48+01:00
29                 B    IV Antibiotics  2014-12-21 12:33:55+01:00

In [54]:
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  c +=1
print(c)

16


In [0]:
act_rename = ["A01","A02","A03","A04","A05","A06","A07","A08","A09","A10","A11","A12","A13","A14","A15","A16"]                     
act_dic = {}
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  act_dic[i] = act_rename[c]
  c +=1
  
def activity_change(row):
  rename = act_dic[row]
  return rename
df_clean["concept:name"] = df_clean["concept:name"].apply(activity_change)

In [56]:
act_dic

{'Admission IC': 'A12',
 'Admission NC': 'A09',
 'CRP': 'A03',
 'ER Registration': 'A01',
 'ER Sepsis Triage': 'A06',
 'ER Triage': 'A05',
 'IV Antibiotics': 'A08',
 'IV Liquid': 'A07',
 'LacticAcid': 'A04',
 'Leucocytes': 'A02',
 'Release A': 'A10',
 'Release B': 'A13',
 'Release C': 'A14',
 'Release D': 'A15',
 'Release E': 'A16',
 'Return ER': 'A11'}

In [0]:
def time_change(row):
  
  if len(row) == 32:
    dateTime = row[:19]
    zone = row[26:]
    newTime = dateTime+zone
    return newTime
  else:
    return row

df_clean["time:timestamp"] = df_clean["time:timestamp"].apply(time_change)
df_clean["time:timestamp"] = pd.to_datetime(df_clean["time:timestamp"],)
df_clean["case:concept:name"] = df_clean["case:concept:name"].astype(str)

In [0]:
small_data = df_clean.iloc[:254,:]

In [34]:
small_data.head(10)

case:concept:name concept:name             time:timestamp
0                 A          A01  2014-10-22 11:15:41+02:00
1                 A          A02  2014-10-22 11:27:00+02:00
2                 A          A03  2014-10-22 11:27:00+02:00
3                 A          A04  2014-10-22 11:27:00+02:00
4                 A          A05  2014-10-22 11:33:37+02:00
5                 A          A06  2014-10-22 11:34:00+02:00
6                 A          A07  2014-10-22 14:03:47+02:00
7                 A          A08  2014-10-22 14:03:47+02:00
8                 A          A09  2014-10-22 14:13:19+02:00
9                 A          A03  2014-10-24 09:00:00+02:00

In [0]:
# Converting csv file to log file
log = conversion_factory.apply(small_data)

# Saving log into xes file
xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile.xes'
xes_exporter.export_log(log, xes_file)

## Counting number of distinct trace

In [0]:
# Save all trace id and activity belongs to a trace into dictionary
def activity(df_eve,unique_id):
  
  traces = {}
  for i in unique_id:
    list_a = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    for n in range(len(row_filter)):
      list_a.append(row_filter.loc[row_filter['case:concept:name']==i,'concept:name'][n])
    traces[i] = list_a
    
  return traces


In [0]:
# Filtering Distinct trace only

def dist_trace(df_eve,unique_id):
  
  Distinct_trace = []
  Distinct_trace_id = []
  
  # Calling activity method
  traces_list = activity(df_clean,unique_id)
  
  for k, v in traces_list.items():
      if v not in Distinct_trace:
        Distinct_trace.append(v)
        Distinct_trace_id.append(k)
        
  return Distinct_trace_id,Distinct_trace


In [0]:
unique_id = df_clean["case:concept:name"].unique()
dist_trace_id, dist_trace = dist_trace(df_clean,unique_id)

In [0]:
# The following functions gets combination of events
activity_name = act_rename

Activity_Name = []
for i in range(len(activity_name)):
  for c in range(len(activity_name)):
    Activity_Name.append([activity_name[i],activity_name[c]])
    #Activity_Name.append([activity_name[c],activity_name[i]])

Activity_combination = []
for combination in Activity_Name:
  if combination not in Activity_combination:
    Activity_combination.append(combination)

In [0]:
len(Activity_combination)

256

In [0]:
for c in Activity_combination:
  print(c)

In [0]:
def combination_counter(trace_list):
  counts_of_combination = {}

  for trace_activity in trace_list:
  
    Activity_name = []
    for i in range(len(trace_activity)-1):
        Activity_name.append([trace_activity[i],trace_activity[i+1]])

    for c in Activity_name:
      if c in Activity_combination:
        k =  repr(c)
        if k not in counts_of_combination.keys():
          counts_of_combination[k] = 1
        else:
          counts_of_combination[k] += 1
        
  return counts_of_combination

In [0]:
CountsofComb = combination_counter(dist_trace)

In [0]:
import operator

#sorted_d = dict( sorted(CountsofComb.items(), key=operator.itemgetter(1),reverse=True))
for k , v in CountsofComb.items():
  print(k, ":", v)

In [17]:
# Frequency of single activity
def activity_count(df_eve,unique_id,fre_act):
  
  Freq_act = 0
  
  for i in unique_id:
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    if fre_act:
      len_act = len(row_filter.loc[row_filter["concept:name"]==fre_act,"concept:name"])
      Freq_act +=len_act
    
  return Freq_act

freq_activity = {}
for i in act_rename:
  frequency_act = activity_count(df_clean,unique_id,i)
  freq_activity[i] = frequency_act
  print("Frequency of {}:{}".format(i,frequency_act))

Frequency of A01:1050
Frequency of A02:3383
Frequency of A03:3262
Frequency of A04:1466
Frequency of A05:1053
Frequency of A06:1049
Frequency of A07:753
Frequency of A08:823
Frequency of A09:1182
Frequency of A10:671
Frequency of A11:294
Frequency of A12:117
Frequency of A13:56
Frequency of A14:25
Frequency of A15:24
Frequency of A16:6


In [0]:
  ab_a = {}
  ab_b = {}
  similar = {}

  # finding the probability of activity given a
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[2:5]]),3)  
      if prob >= 0.5:
        a = repr([k[2:5],k[9:12]])
        ab_a[a] = prob
        #print(k, ":", prob)
  #print()

  # finding the probability of acitivity given b
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[9:12]]),3)
      if prob >= 0.5:
        b = repr([k[2:5],k[9:12]])
        ab_b[b] = prob
        #print(k, ":", prob)      

  for i in ab_a.keys():
    if i in ab_b.keys():
      c = repr([i[2:5],i[9:12]])
      prob_add = ab_a[i] + ab_b[i]
      similar[c] = prob_add 

  similar = dict( sorted(similar.items(), key=operator.itemgetter(1),reverse=True))
  similar_keys = list(similar.keys())
  similar_values = similar.values()

In [19]:
similar_keys[0]

"['A01', 'A05']"

In [0]:
print("Combination :", len(Activity_combination))


Combination : 576


In [0]:
def event_dropper(df_eve,activity_name):
   
  unique_id = df_eve["case:concept:name"].unique()
  df_final = pd.DataFrame(columns=df_eve.columns)
  
  for i in unique_id: 
    event_index = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
      
    
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      
      if activity_name[0] == activity_name[1]:
        if (activity_name[0] == row_filter.loc[event,'concept:name']) & (activity_name[1] == row_filter.loc[event + 1,'concept:name']):
          event_index.append(event)
        
      elif (activity_name[0] == row_filter.loc[event,'concept:name']) & (activity_name[1] == row_filter.loc[event + 1,'concept:name']):
        event_index.append(event)
        row_filter.loc[event + 1,'concept:name']=row_filter.loc[event + 1,'concept:name'].replace(activity_name[1],"D")
       

    row_filter.drop(row_filter.index[event_index],inplace=True)
    df_final = pd.concat([df_final, row_filter],ignore_index=True)
          
  return df_final

In [0]:
replace_activity = {}


In [0]:
df = event_dropper(df_clean, ['A01', 'A05'])

if activity_name[0] = activity_name[1]:
  for i in ['A01', 'A05']: 
    if i not in replace_activity.keys():
      replace_activity[i] = "B"



In [0]:
df = event_dropper(df, ['A05', 'A06'])

for i in ['A05', 'A06']: 
  if i not in replace_activity.keys():
    replace_activity[i] = "C"

In [0]:
df = event_dropper(df, ['A07', 'A08'])

for i in ['A07', 'A08']: 
  if i not in replace_activity.keys():
    replace_activity[i] = "D"

In [28]:
df["concept:name"].unique()

array(['A01', 'A02', 'A03', 'A04', 'C', 'D', 'A09', 'A10', 'B', 'A06',
       'A11', 'A07', 'A08', 'A12', 'A13', 'A14', 'A05', 'A15', 'A16'],
      dtype=object)

In [0]:
for i in replace_activity.keys():
  df["concept:name"] = df["concept:name"].apply(lambda x: re.sub( i, replace_activity[i] , x))


In [31]:
df["concept:name"].unique()

array(['B', 'A02', 'A03', 'A04', 'C', 'D', 'A09', 'A10', 'A11', 'A12',
       'A13', 'A14', 'A15', 'A16'], dtype=object)

In [32]:
len(df["concept:name"].unique())

14

In [34]:
df.head(10)

case:concept:name concept:name             time:timestamp
0                 A            B  2014-10-22 11:15:41+02:00
1                 A          A02  2014-10-22 11:27:00+02:00
2                 A          A03  2014-10-22 11:27:00+02:00
3                 A          A04  2014-10-22 11:27:00+02:00
4                 A            C  2014-10-22 11:34:00+02:00
5                 A            D  2014-10-22 14:03:47+02:00
6                 A          A09  2014-10-22 14:13:19+02:00
7                 A          A03  2014-10-24 09:00:00+02:00
8                 A          A02  2014-10-24 09:00:00+02:00
9                 A          A02  2014-10-26 08:00:00+01:00

In [0]:
# Converting csv file to log file
log = conversion_factory.apply(df)

# Saving log into xes file
xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/BPIC14_2.xes'
xes_exporter.export_log(log, xes_file)

a, b, c  x,y   D c    (a,a) = 0.9  - E

( (a, b,)  (a, c,)  b,  (d, a,)  b) = ( Z, Y, Z, X, Z)               (d,a =.9 - X)  (a,c = .5 - Y)  (a,b = .3 - Z) (c,b =.2 - W)

In [0]:
#Reading XES fileafter 
filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/TestFile.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need11.csv"
csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_sepsis1 = pd.read_csv(fileName)

In [41]:
df_sepsis1.loc[]

case:concept:name concept:name             time:timestamp
0                  A          A01  2014-10-22 11:15:41+02:00
1                  A          A02  2014-10-22 11:27:00+02:00
2                  A          A03  2014-10-22 11:27:00+02:00
3                  A          A04  2014-10-22 11:27:00+02:00
4                  A          A05  2014-10-22 11:33:37+02:00
5                  A          A06  2014-10-22 11:34:00+02:00
6                  A          A07  2014-10-22 14:03:47+02:00
7                  A          A08  2014-10-22 14:03:47+02:00
8                  A          A09  2014-10-22 14:13:19+02:00
9                  A          A03  2014-10-24 09:00:00+02:00
10                 A          A02  2014-10-24 09:00:00+02:00
11                 A          A02  2014-10-26 08:00:00+01:00
12                 A          A03  2014-10-26 08:00:00+01:00
13                 A          A02  2014-10-28 08:00:00+01:00
14                 A          A03  2014-10-28 08:00:00+01:00
15                 A          A03  2014-10-30 08:00:00+01:00
16                 A          A02  2014-10-30 08:00:00+01:00
17                 A          A02  2014-10-31 08:00:00+01:00
18                 A          A03  2014-10-31 08:00:00+01:00
19                 A          A03  2014-11-02 08:00:00+01:00
20                 A          A02  2014-11-02 08:00:00+01:00
21                 A          A10  2014-11-02 15:15:00+01:00
22                 B          A01  2014-12-21 11:04:24+01:00
23                 B          A05  2014-12-21 11:17:19+01:00
24                 B          A03  2014-12-21 11:36:00+01:00
25                 B          A04  2014-12-21 11:36:00+01:00
26                 B          A02  2014-12-21 11:36:00+01:00
27                 B          A06  2014-12-21 12:15:45+01:00
28                 B          A07  2014-12-21 12:33:48+01:00
29                 B          A08  2014-12-21 12:33:55+01:00

In [0]:
unique_id1 = df_sepsis1["case:concept:name"].unique()
dist_trace_id1, dist_trace1 = dist_trace(df_sepsis1,unique_id1)

In [46]:
len(dist_trace1)

20